<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5928c42a2997119173418f7bdca2ccbd0f2aab1f0c70eb7934bd12752f6bc626
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-10 16:25:25
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: -18.32 K (-0.13%)
Current PnL: -29.31 L (-18.93%)
CY Booked + Current PnL: -15.02 L (-9.7%)
-------------------
Total profit:  1.19 L
Total loss:  -30.50 L
-------------------
Total Booked + Current PnL: 11.96 L (9.36%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.28%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.94 L (67.6%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.77,-17.40,21.17,0.09,17623.0,-17533.0,83244.0,93.17,37.0,M-SC,2.35,253.0,-0.99,0.62,9.36,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,2.14,-12.87,14.79,0.02,22495.0,-22461.0,152098.0,132.14,51.0,M-SC,16.61,234.0,-1.00,1.13,32.30,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.74,-2.22,24.82,22.05,23779.0,-2175.0,95805.0,-15.54,56.0,H-SC,6.87,164.0,-0.09,0.71,39.26,XR,ATH,FINANCE
43,ITC,452.00,0.52,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,47.0,X-LC,1.71,5.0,-0.13,1.47,3.98,X40,NTT,FMCG
82,VOLTAS,1530.00,0.37,4.90,14.10,19.69,28360.0,9393.0,201135.0,-5.36,45.0,X-MC,2.57,78.0,0.33,1.50,12.03,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.0,3.46,-35.75,55.63,-0.01,45679.0,-45692.0,82113.0,16704.35,25.0,M-SC,4.09,233.0,-1.00,0.61,5.10,OX40N,NTT,AC
62,SATIN,274.0,3.12,-21.57,93.60,51.83,202814.0,-59602.0,216682.0,-30.35,37.0,H-SC,1.94,146.0,-0.29,1.62,5.56,XY24,NTT,FINANCE
36,ICICIPRULI,790.0,3.09,6.57,22.89,30.97,43114.0,11617.0,188355.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.41,19.96,X40,ATH,INSURANCE
30,HATHWAY,31.4,2.56,-29.14,144.55,73.29,111361.0,-31680.0,77040.0,794.93,43.0,H-SC,7.31,172.0,-0.28,0.57,3.88,XR,NTT,ENTERTAINMENT
18,COFFEEDAY,80.0,2.49,-44.98,145.85,35.27,91123.0,-51072.0,62477.0,-57.75,28.0,L-SC,10.22,270.0,-0.56,0.47,52.91,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-8.63,-20.68,53.28,21.58,59226.0,-28980.0,111159.0,-60.04,16.0,X-MC,6.36,56.0,-0.49,0.83,0.00,X40N,ATH,IT
61,ROUTE,2227.21,-2.91,-49.03,231.68,69.05,157130.0,-65244.0,67822.0,-58.67,45.0,H-SC,23.22,140.0,-0.42,0.51,4.60,SR,ATH,IT
57,RAJOOENG,143.10,-2.55,-37.25,122.48,39.61,78779.0,-38180.0,64320.0,-57.57,23.0,H-SC,26.26,135.0,-0.48,0.48,0.00,AR,ATH,MISC
74,TITAGARH,1548.00,-2.46,-28.93,105.07,45.74,180832.0,-70071.0,172106.0,-5.35,26.0,H-SC,4.04,174.0,-0.39,1.28,11.94,XY24,NTT,ENGINEERING
23,DMART,5391.45,-2.40,-9.31,41.27,28.12,78749.0,-19592.0,190815.0,-21.75,30.0,X-LC,4.63,19.0,-0.25,1.42,12.11,X40N,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.95,-6.20,121.65,107.91,168476.0,-9152.0,138492.0,-23.79,41.0,M-SC,12.15,216.0,-0.05,1.03,3.26,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.14,-12.87,14.79,0.02,22495.0,-22461.0,152098.0,132.14,51.0,M-SC,16.61,234.0,-1.00,1.13,32.30,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.77,-17.40,21.17,0.09,17623.0,-17533.0,83244.0,93.17,37.0,M-SC,2.35,253.0,-0.99,0.62,9.36,OX40N,NTT,DURABLES
66,SIS,528.00,-0.29,-24.88,61.42,21.26,51230.0,-27622.0,83410.0,1966.84,46.0,H-SC,4.45,166.0,-0.54,0.62,12.70,OX40N,NTT,MISC
47,KANSAINER,340.00,1.22,-25.03,51.37,13.47,103849.0,-67509.0,202158.0,-69.09,32.0,H-SC,3.65,158.0,-0.65,1.51,2.79,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.22,5.81,62.8,72.27,112598.0,9848.0,179296.0,-9.26,65.0,M-LC,2.09,99.0,0.09,1.34,12.35,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.22,5.81,62.80,72.27,112598.0,9848.0,179296.0,-9.26,65.0,M-LC,2.09,99.0,0.09,1.34,12.35,XR,NTT,IT
38,INDIAMART,4810.62,-1.02,-3.97,115.27,106.72,136525.0,-4897.0,118439.0,-52.54,31.0,H-SC,1.59,139.0,-0.04,0.88,18.48,AR,ATH,MISC
25,FINCABLES,1641.55,1.95,-6.20,121.65,107.91,168476.0,-9152.0,138492.0,-23.79,41.0,M-SC,12.15,216.0,-0.05,1.03,3.26,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.36,-2.60,39.26,35.63,80136.0,-5458.0,204116.0,-16.30,38.0,H-MC,3.17,119.0,-0.07,1.52,13.63,AR,ATH,PHARMA
50,MASFIN,398.61,0.74,-2.22,24.82,22.05,23779.0,-2175.0,95805.0,-15.54,56.0,H-SC,6.87,164.0,-0.09,0.71,39.26,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-8.63,-20.68,53.28,21.58,59226.0,-28980.0,111159.0,-60.04,16.0,X-MC,6.36,56.0,-0.49,0.83,0.00,X40N,ATH,IT
75,TMPV,600.00,-0.38,-18.63,74.72,42.18,168065.0,-51483.0,224927.0,-26.74,25.0,X-LC,5.62,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.73,-21.29,88.61,48.45,139842.0,-42689.0,157817.0,-26.96,26.0,X-MC,11.68,64.0,-0.31,1.18,2.75,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-1.34,-10.36,40.63,26.07,59637.0,-16960.0,146780.0,-34.01,27.0,X-MC,11.89,55.0,-0.28,1.10,0.00,X40,ATH,APPARELS
33,HINDUNILVR,2922.00,-0.21,-10.46,26.95,13.67,62031.0,-26893.0,230170.0,-17.11,29.0,X-LC,2.79,14.0,-0.43,1.72,8.66,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,0.45,-21.78,96.24,53.50,227153.0,-65723.0,236028.0,-62.10,33.0,X-MC,1.34,58.0,-0.29,1.76,4.84,XY24,NTT,FMCG
15,CAMS,4762.0,-1.64,-80.68,545.96,24.79,265637.0,-203201.0,48655.0,-81.54,36.0,X-SC,1.48,86.0,-0.76,0.36,18.94,X40N,NTT,MISC
43,ITC,452.0,0.52,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,47.0,X-LC,1.71,5.0,-0.13,1.47,3.98,X40,NTT,FMCG
36,ICICIPRULI,790.0,3.09,6.57,22.89,30.97,43114.0,11617.0,188355.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.41,19.96,X40,ATH,INSURANCE
3,ACC,3906.0,-0.15,-25.10,119.16,64.15,212379.0,-59721.0,178230.0,-56.31,35.0,X-SC,1.98,82.0,-0.28,1.33,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.15,-25.10,119.16,64.15,212379.0,-59721.0,178230.0,-56.31,35.0,X-SC,1.98,82.0,-0.28,1.33,0.0,XY24,BTT,CEMENT
22,DIXON,18931.72,-8.63,-20.68,53.28,21.58,59226.0,-28980.0,111159.0,-60.04,16.0,X-MC,6.36,56.0,-0.49,0.83,0.0,X40N,ATH,IT
31,HAVELLS,2069.16,-2.11,-11.01,48.87,32.48,142641.0,-36112.0,291879.0,-15.47,34.0,X-MC,5.34,67.0,-0.25,2.18,0.0,X40,ATH,ELECTRICAL
53,PAGEIND,51605.07,-1.34,-10.36,40.63,26.07,59637.0,-16960.0,146780.0,-34.01,27.0,X-MC,11.89,55.0,-0.28,1.10,0.0,X40,ATH,APPARELS
75,TMPV,600.00,-0.38,-18.63,74.72,42.18,168065.0,-51483.0,224927.0,-26.74,25.0,X-LC,5.62,3.0,-0.31,1.68,0.0,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.22,-29.81,103.51,42.84,47227.0,-19380.0,45626.0,-54.27,41.0,X-SC,36.81,83.0,-0.41,0.34,3.23,XY24,NTT,MISC
15,CAMS,4762.00,-1.64,-80.68,545.96,24.79,265637.0,-203201.0,48655.0,-81.54,36.0,X-SC,1.48,86.0,-0.76,0.36,18.94,X40N,NTT,MISC
59,RELAXO,1176.00,-2.19,-46.68,185.64,52.31,143685.0,-67760.0,77400.0,-43.28,46.0,X-SC,6.16,91.0,-0.47,0.58,3.60,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.19,-37.27,115.53,35.20,93254.0,-47952.0,80718.0,5.85,37.0,X-SC,15.38,92.0,-0.51,0.60,2.53,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-8.63,-20.68,53.28,21.58,59226.0,-28980.0,111159.0,-60.04,16.0,X-MC,6.36,56.0,-0.49,0.83,0.00,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-0.60,-13.35,37.65,19.28,112869.0,-46171.0,299785.0,-24.60,59.0,X-LC,5.49,1.0,-0.41,2.24,10.83,X40,ATH,IT
41,INFY,2275.00,-0.89,7.89,43.55,54.87,147008.0,24674.0,337562.0,-16.01,60.0,X-LC,3.28,2.0,0.17,2.52,16.82,X40,BTT,IT
75,TMPV,600.00,-0.38,-18.63,74.72,42.18,168065.0,-51483.0,224927.0,-26.74,25.0,X-LC,5.62,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
81,VBL,671.64,0.30,-4.46,42.01,35.67,126762.0,-14100.0,301742.0,-15.77,54.0,X-LC,4.72,4.0,-0.11,2.25,8.77,X40N,ATH,FMCG
43,ITC,452.00,0.52,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,47.0,X-LC,1.71,5.0,-0.13,1.47,3.98,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,2.19,-17.78,120.97,81.67,93302.0,-16682.0,77128.0,6927.78,40.0,L-SC,14.29,271.0,-0.18,0.58,49.36,XR,NTT,CERAMICS
50,MASFIN,398.61,0.74,-2.22,24.82,22.05,23779.0,-2175.0,95805.0,-15.54,56.0,H-SC,6.87,164.0,-0.09,0.71,39.26,XR,ATH,FINANCE
13,BSOFT,831.70,-1.70,-23.29,96.57,50.79,104189.0,-32763.0,107890.0,-1.43,65.0,H-SC,5.81,151.0,-0.31,0.81,25.62,XR,ATH,IT
39,INDIGOPNTS,1408.00,2.14,-12.87,14.79,0.02,22495.0,-22461.0,152098.0,132.14,51.0,M-SC,16.61,234.0,-1.00,1.13,32.30,OX40N,NTT,PAINTS
36,ICICIPRULI,790.00,3.09,6.57,22.89,30.97,43114.0,11617.0,188355.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.41,19.96,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-1.70,-23.29,96.57,50.79,104189.0,-32763.0,107890.0,-1.43,65.0,H-SC,5.81,151.0,-0.31,0.81,25.62,XR,ATH,IT
32,HCLTECH,1922.01,0.51,8.12,15.37,24.73,40202.0,19638.0,261562.0,12.76,63.0,X-LC,5.98,7.0,0.49,1.95,24.59,X40,ATH,IT
84,WIPRO,420.00,0.22,5.81,62.80,72.27,112598.0,9848.0,179296.0,-9.26,65.0,M-LC,2.09,99.0,0.09,1.34,12.35,XR,NTT,IT
56,RAJESHEXPO,518.00,0.96,-63.25,172.30,0.07,86858.0,-86766.0,50411.0,1747.22,55.0,L-SC,2.31,268.0,-1.00,0.38,25.37,OX40N,NTT,JEWELLERY
39,INDIGOPNTS,1408.00,2.14,-12.87,14.79,0.02,22495.0,-22461.0,152098.0,132.14,51.0,M-SC,16.61,234.0,-1.00,1.13,32.30,OX40N,NTT,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.71
1,25,45.79
2,50,76.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.04
MC    30.31
LC    24.66
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.33
X40      22.49
X40N     10.82
XR       10.08
XY25      9.40
AR        9.32
OX40N     7.80
SR        1.03
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.47
X-MC    23.56
X-LC    21.07
M-SC    12.15
X-SC     6.99
H-MC     4.99
L-SC     1.43
M-MC     1.41
M-LC     1.34
H-LC     1.19
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.40,-7.91,42.81
IT,12.71,-18.34,82.32
FINANCE,9.57,-21.02,72.15
ELECTRICAL,6.24,-12.48,53.68
PAINTS,6.14,-12.13,28.79
MISC,6.06,-59.60,124.98
INSURANCE,4.65,-1.42,37.16
PHARMA,4.11,-1.31,33.73
AUTO,3.40,-26.33,80.50


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3240993.0,21
AR,1322444.0,10
XR,1311211.0,13
X40,1025264.0,14
X40N,998325.0,9
OX40N,744994.0,10
XY25,382890.0,6
SR,282781.0,2
MH,84990.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3696609.0,25
M-SC,1429480.0,15
X-MC,1326429.0,16
X-SC,984108.0,8
X-LC,939222.0,11
H-MC,409645.0,3
L-SC,271283.0,3
M-LC,112598.0,1
H-LC,74585.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1290775.0      6
           AR         904905.0      5
M-SC       XY24       816108.0      6
H-SC       XR         784947.0      7
X-SC       X40N       506489.0      3
X-MC       X40        460457.0      7
           XY24       392898.0      3
X-LC       X40        381673.0      5
H-SC       OX40N      348211.0      4
M-SC       OX40N      309925.0      5
X-SC       XY24       294485.0      3
X-MC       X40N       286325.0      4
H-SC       SR         282781.0      2
X-LC       XY24       250758.0      2
H-MC       AR         213676.0      2
X-LC       X40N       205511.0      2
H-MC       XY24       195969.0      1
X-MC       XY25       186749.0      2
L-SC       XR         184425.0      2
X-SC       X40        183134.0      2
M-SC       XR         174169.0      2
           AR         129278.0      2
M-LC       XR         112598.0      1
X-LC       XY25       101280.0      2
L-SC       OX40N       86858.0      1
H-SC       MH          84990.0      1
H-LC       AR          74585.0      1
M-MC       XY25        56868.0      1
L-MC       XR          55072.0      1
L-LC       XY25        37993.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
